<a href="https://colab.research.google.com/github/EvenSol/NeqSim-Colab/blob/master/notebooks/reservoir/fieldDevelopment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Use of NeqSim for field development analysis
In this notebokk we will analyze a field development using neqsim. The analysis will be from reservoir to market.

In [1]:
%%capture
!pip install neqsim

# Creating reservoir fluid

In [2]:
import neqsim
from neqsim.thermo.thermoTools import *
from neqsim.process.processTools import simplereservoir,clearProcess,runProcess, getProcess


fluid1 = fluid('pr')
fluid1.addComponent("nitrogen", 0.100)
fluid1.addComponent("CO2", 0.100)
fluid1.addComponent("methane", 30.00)
fluid1.addComponent("ethane", 1.0)
fluid1.addComponent("propane", 1.0)
fluid1.addComponent("i-butane", 1.0)
fluid1.addComponent("n-butane", 1.0)
fluid1.addComponent("n-hexane", 0.1)
fluid1.addComponent("n-heptane", 0.1)
fluid1.addComponent("n-nonane", 1.0)
fluid1.addComponent("nC10", 1.0)
fluid1.addComponent("nC12", 3.0)
fluid1.addComponent("nC15", 3.0)
fluid1.addComponent("nC20", 1.0)
fluid1.addComponent("water", 11.0)
fluid1.setMixingRule(2)
fluid1.setMultiPhaseCheck(True)

# Create reservoir and wells

In [14]:
clearProcess()

fluid1.setPressure(100.0, 'bara')
fluid1.setTemperature(100.0, 'C')
reservoir1Ops = simplereservoir(fluid1, "Reservoir Formation 1", gasvolume=5.0 * 1e7, oilvolume=552.0 * 1e6, watervolume=10.0e6)

initFLowRate = 25020000.0
producedOilStream1 = reservoir1Ops.addOilProducer("SLP_A32562O")
producedOilStream1.setFlowRate(initFLowRate, "kg/day")

fluid2 = fluid1.clone()
fluid2.addComponent('methane', 10.0);
fluid2.setPressure(100.0, 'bara')
fluid2.setTemperature(90.0, 'C')
reservoir2Ops = simplereservoir(fluid2, "Reservoir Formation 2", gasvolume=5.0 * 1e8, oilvolume=52.0 * 1e6, watervolume=10.0e6)

producedGasStream2 = reservoir2Ops.addGasProducer("SLP_A32562O")
producedGasStream2.setFlowRate(7.0, "MSm3/day")

runProcess()
getProcess().save('reservoir.neqsim')


# Calculate well flow rates
Insert Beggs and Brills multiphase calculation for well stream... and influx equiations for wells...

In [21]:
from neqsim.process import getProcess, openprocess
resmodel = openprocess('reservoir.neqsim')
reservoir1 = resmodel.getUnit('Reservoir Formation 1')
wellStream1 = reservoir1.getOilProducer(0).getStream()
wellStream1.setFlowRate(initFLowRate, "kg/day")

reservoir2 = resmodel.getUnit('Reservoir Formation 2')
wellStream2 = reservoir2.getGasProducer(0).getStream()

deltat = 24*60*60.0*365
resmodel.setTimeStep(deltat)

resmodel.runTransient()

print('pressure res 1 ', reservoir1.getReservoirFluid().getPressure('bara'))
print('pressure res 2 ', reservoir2.getReservoirFluid().getPressure('bara'))

pressure res 1  95.28551722459422
pressure res 2  94.79809143790816


# Add subsea system

In [ ]:
from neqsim.process.processTools import mixer,cooler

template1 = mixer("template 1")
template1.addStream(wellStream1)
template1.addStream(wellStream2)

template1.run()


pipeline = cooler(template1.getOutStream())
pipeline.setOutPressure(85.0, 'bara')
pipeline.setOutTemperature(45.0, 'C')

pipeline.run()

printFrame(pipeline.getOutStream().getFluid())


#print('gas flow rate ', 



| 0                  | 1           | 2          | 3           | 4           | 5   | 6               |
|:-------------------|:------------|:-----------|:------------|:------------|:----|:----------------|
|                    | total       | gas        | oil         | aqueous     |     |                 |
| nitrogen           | 2.20418E-3  | 3.6422E-3  | 6.35498E-4  | 3.28952E-9  |     | [mole fraction] |
| CO2                | 2.14563E-3  | 2.3016E-3  | 2.05536E-3  | 9.74335E-7  |     | [mole fraction] |
| methane            | 6.48547E-1  | 9.42235E-1 | 3.36485E-1  | 1.90326E-7  |     | [mole fraction] |
| ethane             | 2.16347E-2  | 2.08132E-2 | 2.34902E-2  | 4.01918E-11 |     | [mole fraction] |
| propane            | 2.2196E-2   | 1.30954E-2 | 3.36383E-2  | 6.18221E-13 |     | [mole fraction] |
| i-butane           | 2.27436E-2  | 8.76209E-3 | 3.98468E-2  | 1.31224E-15 |     | [mole fraction] |
| n-butane           | 2.29588E-2  | 7.24962E-3 | 4.20667E-2  | 1.61785E-15 |     

# Setting up topside process


In [ ]:
from neqsim.process.processTools import separator3phase

inletSeparator = separator3phase(pipeline.getOutStream())
inletSeparator.setName("1st stage separator")

inletSeparator.run()


print('gas flow ', inletSeparator.getGasOutStream().getFlowRate('MSm3/day'), ' MSm3/day')
print('oil flow ', inletSeparator.getOilOutStream().getFlowRate('kg/day')/inletSeparator.getOilOutStream().getFluid().getPhase('oil').getDensity('kg/m3'), ' m3/day')

gas flow  6.520314669832758  MSm3/day
oil flow  38276.84387956223  m3/day
